## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-31-22-47-00 +0000,wsj,Rudy Giuliani Injured in Car Crash in New Hamp...,https://www.wsj.com/us-news/rudy-giuliani-inju...
1,2025-08-31-22-46-25 +0000,nyt,Judge Halts U.S. Effort to Deport Guatemalan C...,https://www.nytimes.com/2025/08/31/us/politics...
2,2025-08-31-22-42-23 +0000,nyt,Brazil Tightens Bolsonaro House Arrest Amid Es...,https://www.nytimes.com/2025/08/31/world/ameri...
3,2025-08-31-22-37-00 +0000,wsj,Federal Judge Blocks Deportations of Guatemala...,https://www.wsj.com/us-news/law/federal-judge-...
4,2025-08-31-22-33-24 +0000,startribune,Twins top Padres to win series behind strong g...,https://www.startribune.com/twins-top-padres-t...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,26
72,china,11
78,will,9
5,new,9
96,shooting,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
62,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...,63
72,2025-08-31-17-11-30 +0000,nypost,"Trump says he, Putin and Zelensky will meet to...",https://nypost.com/2025/08/31/world-news/trump...,60
38,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...,58
77,2025-08-31-16-15-56 +0000,nypost,Rep. Ilhan Omar slams Trump official for point...,https://nypost.com/2025/08/31/us-news/rep-ilha...,56
25,2025-08-31-21-23-44 +0000,nypost,Israel begins heavy airstrikes ahead of Gaza C...,https://nypost.com/2025/08/31/world-news/israe...,55


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
62,63,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...
16,46,2025-08-31-21-49-36 +0000,nypost,Bullet fragment found in neck of Minneapolis s...,https://nypost.com/2025/08/31/us-news/bullet-f...
25,46,2025-08-31-21-23-44 +0000,nypost,Israel begins heavy airstrikes ahead of Gaza C...,https://nypost.com/2025/08/31/world-news/israe...
49,40,2025-08-31-19-00-45 +0000,nypost,Rudy Giuliani seriously injured in New Hampshi...,https://nypost.com/2025/08/31/us-news/rudy-giu...
109,31,2025-08-31-10-00-11 +0000,wapo,Putin heads to China to shore up trading partn...,https://www.washingtonpost.com/world/2025/08/3...
77,30,2025-08-31-16-15-56 +0000,nypost,Rep. Ilhan Omar slams Trump official for point...,https://nypost.com/2025/08/31/us-news/rep-ilha...
130,25,2025-08-31-09-00-00 +0000,wsj,Democratic Party moderates see Virginia govern...,https://www.wsj.com/politics/elections/in-virg...
38,23,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...
145,21,2025-08-31-02-17-54 +0000,nypost,Lisa Cook standing up for the Federal Reserve ...,https://nypost.com/2025/08/30/business/lisa-co...
68,21,2025-08-31-17-28-26 +0000,nypost,AI chatbot law pitched by NYC pol after ‘delus...,https://nypost.com/2025/08/31/us-news/nyc-pol-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
